In [5]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Import Rental Data


In [6]:
rental_data = pd.read_csv("../data/raw/rental_data.csv")

In [7]:
rental_data

,lat,lng,address,bed,bath,car,type,rented_price,date
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01
1,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01
...,...,...,...,...,...,...,...,...,...
3388731,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-12-01
3388732,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-11-01
3388733,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,280,2009-11-01
3388734,-37.895171,144.759854,"53 BROADBEACH CIRCUIT, POINT COOK",4.0,2.0,2.0,Townhouse,420,2019-09-01


In [8]:
# get a new column year from the rental dataframe
rental_data['year'] = pd.DatetimeIndex(rental_data['date']).year
rental_data

,lat,lng,address,bed,bath,car,type,rented_price,date,year
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01,2023
1,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01,2023
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01,2023
...,...,...,...,...,...,...,...,...,...,...
3388731,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-12-01,2011
3388732,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,330,2011-11-01,2011
3388733,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,280,2009-11-01,2009
3388734,-37.895171,144.759854,"53 BROADBEACH CIRCUIT, POINT COOK",4.0,2.0,2.0,Townhouse,420,2019-09-01,2019


Select only years after 2017

In [9]:
rental_data = rental_data[rental_data['year'] >= 2017]

# Import population and income data

In [10]:
population_data = pd.read_csv("../data/raw/population/population-01-23.csv")
income_data_2 = pd.read_csv("../data/raw/income/income-17-21.csv")

population_data

,sa2_code,sa2_name,year,population,geometry
0,201011001,Alfredton,2001,5756.0,POLYGON ((143.78282104711133 -37.5666578080732...
1,201011002,Ballarat,2001,11497.0,POLYGON ((143.81896375194268 -37.5558184903035...
2,201011005,Buninyong,2001,5320.0,POLYGON ((143.8417079819547 -37.61596158212406...
3,201011006,Delacombe,2001,4154.0,POLYGON ((143.7505006002539 -37.59119089617515...
4,201011007,Smythes Creek,2001,3317.0,POLYGON ((143.7329551500814 -37.62333175526272...
...,...,...,...,...,...
12001,217031476,Otway,2023,3983.0,MULTIPOLYGON (((143.40263218222083 -38.7815167...
12002,217041477,Moyne - East,2023,7132.0,POLYGON ((142.41438437646818 -38.0930256782602...
12003,217041478,Moyne - West,2023,10148.0,MULTIPOLYGON (((142.00870118938005 -38.4171452...
12004,217041479,Warrnambool - North,2023,22762.0,POLYGON ((142.4366836580644 -38.35544112113083...


In [11]:
# change the type of income_data_2 to float
income_data_2['median_income'] = income_data_2['median_income'].str.replace(',', '').astype(float)

In [12]:
population_data["geometry"] = population_data["geometry"].apply(wkt.loads)
income_data_2["geometry"] = income_data_2["geometry"].apply(wkt.loads)

gdf_population = gpd.GeoDataFrame(population_data, geometry="geometry", crs="EPSG:4326")
gdf_income_2 = gpd.GeoDataFrame(income_data_2, geometry="geometry", crs="EPSG:4326")

gdf_rental = gpd.GeoDataFrame(rental_data, geometry=gpd.points_from_xy(rental_data["lng"], rental_data["lat"]), crs="EPSG:4326")

# join rental data with population and income

In [13]:
# left sptail join rental lat lon on population geometry column

joined_gdf = gpd.sjoin(gdf_rental,gdf_population, how="left", predicate = "within", on_attribute=["year"])


In [14]:
joined_gdf = joined_gdf.dropna()

Only selecting the data recorded after year 2016

In [15]:
joined_gdf2 = joined_gdf[joined_gdf["year"] > 2016]

In [16]:
joined_gdf2.drop(columns=["index_right"], inplace=True)
joined_gdf_new2 = gpd.sjoin(joined_gdf2,gdf_income_2, how="left", predicate = "within", on_attribute=["year"])
joined_gdf_new2

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code_left,sa2_name_left,population,index_right,sa2_code_right,sa2_name_right,median_income
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,16098.0,2364.0,206041505.0,Melbourne CBD - West,39300.0
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
5,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,650,2023-06-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,20027.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388726,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-09-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388727,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-08-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388728,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-07-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9739.0,1408.0,213051581.0,Point Cook - North West,57433.0
3388729,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,365,2018-06-01,2018,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,9678.0,889.0,213051581.0,Point Cook - North West,57001.0


In [17]:
joined_gdf = joined_gdf_new2

joined_gdf_new2 is the dataframe that has the data from 2021 - 2023

In [18]:
joined_gdf = joined_gdf.dropna()

In [19]:
joined_gdf.drop(columns = "index_right",inplace=True)

C:\Users\Richard\AppData\Local\Temp\ipykernel_31460\3371548854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_gdf.drop(columns = "index_right",inplace=True)


# Import cpidata, unemployement rate and housing interest data

In [20]:
cpi_data = pd.read_csv("../data/curated/cpi_data.csv")
unemployment_data = pd.read_csv("../data/curated/yearly_avg_unemployment.csv")
housing_interest_data = pd.read_csv("../data/curated/housing_interest.csv")

In [21]:
# change the time column to year only
housing_interest_data["time"] = housing_interest_data["time"].apply(lambda x: x[:4])

In [22]:
# aggregate the housing_interest_data by year
housing_interest_data = housing_interest_data.groupby("time")["interest_rate"].mean().reset_index()
housing_interest_data

,time,interest_rate
0,2019,4.171667
1,2020,3.656667
2,2021,3.415000
3,2022,4.308333
4,2023,6.374167
5,2024,6.661667


In [23]:
housing_interest_data.rename(columns={"time":"year"}, inplace=True)

In [24]:
unemployment_data.rename(columns = {"Yearly Average Seasonally Adjusted (%)":"unemployment_rate"}, inplace=True)

In [25]:
cpi_data.drop(columns = ["time","Change from previous quarter (%)"] ,inplace=True)
cpi_data.rename(columns = {"Annual change (%)" : "cpi" }, inplace=True)
cpi_data

,cpi,year
0,1.70,2014
1,1.70,2015
2,1.50,2016
3,1.90,2017
4,1.80,2018
5,1.80,2019
6,0.90,2020
7,3.50,2021
8,7.80,2022
9,5.60,2023


In [26]:
# add a row in unemployment_data that 2021 has rate 4.2
new_row = pd.DataFrame({"year": [2021], "unemployment_rate": [4.2]})
unemployment_data = pd.concat([unemployment_data, new_row], ignore_index=True)
unemployment_data

,year,unemployment_rate
0,2014,6.216667
1,2015,6.058333
2,2016,5.716667
3,2017,5.608333
4,2018,5.283333
5,2019,5.175000
6,2020,5.166667
7,2022,3.611111
8,2023,3.691667
9,2024,4.014286


# model median income using time series

In [27]:
# join the cpi_data and unemployment_data on year
joined_gdf_with_cpi = pd.merge(joined_gdf, cpi_data, on="year", how="left")
joined_gdf_with_cpi_and_unem = pd.merge(joined_gdf_with_cpi, unemployment_data, on="year", how="left")

In [28]:
joined_gdf_with_cpi_and_unem.drop(columns = ["lat","lng","address","sa2_code_left","sa2_name_left","sa2_code_right","date"], inplace=True)
joined_gdf_with_cpi_and_unem.rename(columns={"sa2_name_right":"region"},inplace = True)

In [29]:
# find the unique values of the region column
unique_region = joined_gdf_with_cpi_and_unem["region"].unique()
median_income_pred = pd.DataFrame(columns=["year","region","median_income"])

In [31]:
# Predict using each data for a specific region

for j in range(2022,2030):
    for i in range(0,len(unique_region)):
        
        region_0 = joined_gdf_with_cpi_and_unem[joined_gdf_with_cpi_and_unem["region"] == unique_region[i]]
        X = region_0[["cpi","year","unemployment_rate"]]
        y = region_0["median_income"]

        model = LinearRegression()
        model.fit(X,y)

        data_2022 = [cpi_data[cpi_data["year"] == j]["cpi"].values[0],j,unemployment_data[unemployment_data["year"] == j]["unemployment_rate"].values[0]]

        new_row = pd.DataFrame({"year": [j], "region": [unique_region[i]], "median_income": model.predict([data_2022])[0]})

        median_income_pred = pd.concat([median_income_pred, new_row], ignore_index=True)

median_income_pred.rename(columns={"median_income":"median_pred_income"},inplace=True)
income_data_2.rename(columns={"sa2_name":"region"}, inplace=True)
income_data_2.head(1)

# join the median_income_pred with income_data_2 on region
median_income_pred = pd.merge(median_income_pred, income_data_2, on="region", how="left")

median_income_pred.drop(columns=["year_y","median_income"], inplace=True)
median_income_pred.rename(columns={"year_x":"year"}, inplace=True)

median_income_pred.drop_duplicates(inplace=True)
median_income_pred.rename(columns={"median_pred_income":"median_income"}, inplace=True)

# join the median_income_pred with income_data_2 on region
income_data_full = pd.concat([income_data_2, median_income_pred], ignore_index=True)
income_data_full

C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Richard\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493

,sa2_code,region,geometry,year,median_income
0,206071139,Abbotsford,POLYGON ((144.99254539277322 -37.8024856893544...,2017,58219.000000
1,210011226,Airport West,POLYGON ((144.867064292942 -37.724714940312566...,2017,53423.000000
2,206051128,Albert Park,POLYGON ((144.96766973330057 -37.8373669078686...,2017,65352.000000
3,204011054,Alexandra,POLYGON ((145.59015065379967 -37.2247702673797...,2017,36773.000000
4,201011001,Alfredton,POLYGON ((143.78282104711133 -37.5666578080732...,2017,50596.000000
...,...,...,...,...,...
6710,211031451,Croydon - West,POLYGON ((145.28818275389443 -37.7809559896419...,2029,65238.027082
6711,215031402,Kerang,"POLYGON ((143.89191430851 -35.735210753426436,...",2029,59128.518258
6712,213041574,Fraser Rise - Plumpton,POLYGON ((144.67057300877906 -37.7078414069349...,2029,59227.000000
6713,205041096,Morwell,POLYGON ((146.36524250548774 -38.2250693398236...,2029,40893.000000


create new dataframe that used our prediction

In [32]:
gdf_income_new = gpd.GeoDataFrame(income_data_full, geometry="geometry", crs="EPSG:4326")

In [33]:
final_rental = gpd.sjoin(gdf_rental,gdf_income_new, how="left", predicate = "within", on_attribute=["year"])
final_rental.drop(columns=["index_right"], inplace=True)
final_rental = gpd.sjoin(final_rental,gdf_population, how="left", predicate = "within", on_attribute=["year"])
final_rental = final_rental.dropna()

In [34]:
final_rental = final_rental.merge(cpi_data, on="year", how="left")
final_rental = final_rental.merge(unemployment_data, on="year", how="left")


In [35]:
final_rental

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code_left,region,median_income,index_right,sa2_code_right,sa2_name,population,cpi,unemployment_rate
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,44492.4005,11621.0,206041505.0,Melbourne CBD - West,20027.0,5.6,3.691667
1,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2023-07-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,44492.4005,11621.0,206041505.0,Melbourne CBD - West,20027.0,5.6,3.691667
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,540,2021-08-01,2021,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,39300.0000,10577.0,206041505.0,Melbourne CBD - West,16098.0,3.5,4.200000
3,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2023-08-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,44492.4005,11621.0,206041505.0,Melbourne CBD - West,20027.0,5.6,3.691667
4,-37.813730,144.955580,"1702/560 LONSDALE STREET, MELBOURNE",2.0,1.0,0.0,Unit/apmt,650,2023-06-01,2023,POINT (144.95558 -37.81373),206041505.0,Melbourne CBD - West,44492.4005,11621.0,206041505.0,Melbourne CBD - West,20027.0,5.6,3.691667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531551,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-09-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,57433.0000,9841.0,213051581.0,Point Cook - North West,9739.0,1.8,5.175000
1531552,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-08-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,57433.0000,9841.0,213051581.0,Point Cook - North West,9739.0,1.8,5.175000
1531553,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,370,2019-07-01,2019,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,57433.0000,9841.0,213051581.0,Point Cook - North West,9739.0,1.8,5.175000
1531554,-37.883631,144.732710,"96 BOARDWALK BOULEVARD, POINT COOK",3.0,2.0,2.0,Unit/apmt,365,2018-06-01,2018,POINT (144.73271 -37.88363),213051581.0,Point Cook - North West,57001.0000,9319.0,213051581.0,Point Cook - North West,9678.0,1.8,5.283333


In [36]:
# extract the suburbs from the address column
final_rental["suburb"] = final_rental["address"].apply(lambda x: x.split(", ")[1])

In [37]:
final_rental.to_csv("../data/curated/rental_data_cleaned.csv", index=False)

In [38]:
income_data_full.to_csv("../data/curated/predicted_median_income.csv")

# Population prediction from abs

In [39]:
population_pred = pd.read_csv("../data/landing/population_prediction.csv")


population_pred.dropna(inplace=True)

population_pred["Statistical Areas Level 2 2021 code"] = population_pred["Statistical Areas Level 2 2021 code"].astype(str)
# remove all rows that doesn't start with 2 in Statistical Areas Level 2 2021 code

population_pred = population_pred[population_pred["Statistical Areas Level 2 2021 code"].str.startswith("2")]


C:\Users\Richard\AppData\Local\Temp\ipykernel_31460\1545936041.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  population_pred = pd.read_csv("../data/landing/population_prediction.csv")


In [40]:
# remove all columns other than the Projected persons (total) column
population_pred = population_pred[["Statistical Areas Level 2 2021 code","Projected persons (total)","Statistical Areas Level 2 2021 name","Reference date"]]

In [41]:
population_pred["year"] = population_pred["Reference date"].apply(lambda x: x[5:10])

# find the distinct values of the Statistical Areas Level 2 2021 code
unique_num = population_pred["Statistical Areas Level 2 2021 code"].unique()
len(unique_num)

522

In [42]:
population_pred.to_csv("../data/curated/population_prediction.csv", index=False)

# get the dataset for 2024

In [43]:
rental_2024 = rental_data[rental_data["year"] == 2024]

In [83]:
gpd_rental_2024 = gpd.GeoDataFrame(rental_2024, geometry=gpd.points_from_xy(rental_2024["lng"], rental_2024["lat"]), crs="EPSG:4326")


In [84]:
gpd_rental_2024 = gpd.sjoin(gpd_rental_2024,gdf_income_new, how="left", predicate = "within", on_attribute=["year"])
gpd_rental_2024.drop(columns=["index_right"], inplace=True)


In [85]:
gpd_rental_2024.dropna(inplace=True)
gpd_rental_2024["sa2_code"] = gpd_rental_2024["sa2_code"].astype(int)

In [86]:
population_pred["Statistical Areas Level 2 2021 code"] = population_pred["Statistical Areas Level 2 2021 code"].astype(int)

In [87]:
# joint populationpred with gpd_rental_2024 on sa2_code
gpd_rental_2024 = gpd_rental_2024.merge(population_pred, left_on="sa2_code", right_on="Statistical Areas Level 2 2021 code", how="left")

In [88]:
gpd_rental_2024.drop(columns=["Statistical Areas Level 2 2021 code","Reference date"], inplace=True)

gpd_rental_2024.rename(columns={"Projected persons (total)":"population"}, inplace=True)
gpd_rental_2024

,lat,lng,address,bed,bath,car,type,rented_price,date,year_x,geometry,sa2_code,region,median_income,population,Statistical Areas Level 2 2021 name,year_y
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,17299,Melbourne CBD - West,2022
1,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,19024,Melbourne CBD - West,2023
2,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,20162,Melbourne CBD - West,2024
3,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,21126,Melbourne CBD - West,2025
4,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,21955,Melbourne CBD - West,2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170417,-38.083631,145.450307,"11 PINNACLE DRIVE, PAKENHAM",3.0,2.0,2.0,Unit/apmt,520,2024-08-01,2024,POINT (145.45031 -38.08363),212011552,Pakenham - South West,55683.420715,30238,Pakenham - South West,2028
1170418,-38.083631,145.450307,"11 PINNACLE DRIVE, PAKENHAM",3.0,2.0,2.0,Unit/apmt,520,2024-08-01,2024,POINT (145.45031 -38.08363),212011552,Pakenham - South West,55683.420715,30633,Pakenham - South West,2029
1170419,-38.083631,145.450307,"11 PINNACLE DRIVE, PAKENHAM",3.0,2.0,2.0,Unit/apmt,520,2024-08-01,2024,POINT (145.45031 -38.08363),212011552,Pakenham - South West,55683.420715,31005,Pakenham - South West,2030
1170420,-38.083631,145.450307,"11 PINNACLE DRIVE, PAKENHAM",3.0,2.0,2.0,Unit/apmt,520,2024-08-01,2024,POINT (145.45031 -38.08363),212011552,Pakenham - South West,55683.420715,31357,Pakenham - South West,2031


In [89]:
# only keep the rows that has both year_x and year_y as 2024
gpd_rental_2024 = gpd_rental_2024[gpd_rental_2024["year_x"] == 2024]



In [101]:
gpd_rental_2024["year_y"] = gpd_rental_2024["year_y"].apply((lambda x: int(x[0:5])))
gpd_rental_2024["year_y"][0]

2022

In [103]:
gpd_rental_2024 = gpd_rental_2024[gpd_rental_2024["year_y"] == 2024]

In [107]:
gpd_rental_2024.drop(columns=["year_y"], inplace=True)
gpd_rental_2024.rename(columns={"year_x":"year"}, inplace=True)

C:\Users\Richard\AppData\Local\Temp\ipykernel_31460\3004514086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpd_rental_2024.drop(columns=["year_y"], inplace=True)
C:\Users\Richard\AppData\Local\Temp\ipykernel_31460\3004514086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpd_rental_2024.rename(columns={"year_x":"year"}, inplace=True)


In [109]:
# join gpd_rental_2024 with cpi_data and unemployment_data on year
gpd_rental_2024 = gpd_rental_2024.merge(cpi_data, on="year", how="left")
gpd_rental_2024 = gpd_rental_2024.merge(unemployment_data, on="year", how="left")

gpd_rental_2024

,lat,lng,address,bed,bath,car,type,rented_price,date,year,geometry,sa2_code,region,median_income,population,Statistical Areas Level 2 2021 name,cpi_x,cpi_y,unemployment_rate
0,-37.813730,144.955580,"201/560 LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,730,2024-03-01,2024,POINT (144.95558 -37.81373),206041505,Melbourne CBD - West,42430.858339,20162,Melbourne CBD - West,3.53,3.53,4.014286
1,-37.813590,144.955720,"901/21 HEALEYS LANE, MELBOURNE",2.0,1.0,0.0,Unit/apmt,720,2024-08-01,2024,POINT (144.95572 -37.81359),206041505,Melbourne CBD - West,42430.858339,20162,Melbourne CBD - West,3.53,3.53,4.014286
2,-37.813700,144.953930,"3801/601 LITTLE LONSDALE STREET, MELBOURNE",2.0,2.0,1.0,Unit/apmt,800,2024-04-01,2024,POINT (144.95393 -37.8137),206041505,Melbourne CBD - West,42430.858339,20162,Melbourne CBD - West,3.53,3.53,4.014286
3,-37.809259,144.966567,"1808/318 RUSSELL STREET, MELBOURNE",2.0,2.0,0.0,Unit/apmt,840,2024-02-01,2024,POINT (144.96657 -37.80926),206041503,Melbourne CBD - East,42412.280478,11840,Melbourne CBD - East,3.53,3.53,4.014286
4,-37.852740,144.979740,"707/77 QUEENS ROAD, MELBOURNE",2.0,2.0,1.0,Unit/apmt,680,2024-08-01,2024,POINT (144.97974 -37.85274),206051514,St Kilda - West,66910.824282,15041,St Kilda - West,3.53,3.53,4.014286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106397,-38.069069,145.463520,"7 ROSSETTI COURT, PAKENHAM",3.0,1.0,0.0,House,520,2024-08-01,2024,POINT (145.46352 -38.06907),212011552,Pakenham - South West,55683.420715,28466,Pakenham - South West,3.53,3.53,4.014286
106398,-38.078518,145.482727,"7 DISCOVERY STREET, PAKENHAM",3.0,2.0,2.0,House,500,2024-08-01,2024,POINT (145.48273 -38.07852),212011551,Pakenham - South East,57772.670788,7044,Pakenham - South East,3.53,3.53,4.014286
106399,-38.078518,145.482727,"7 DISCOVERY STREET, PAKENHAM",3.0,2.0,2.0,House,500,2024-07-01,2024,POINT (145.48273 -38.07852),212011551,Pakenham - South East,57772.670788,7044,Pakenham - South East,3.53,3.53,4.014286
106400,-38.052810,145.454962,"19 PORTILLO CRESCENT, PAKENHAM",4.0,2.0,2.0,Unit/apmt,580,2024-08-01,2024,POINT (145.45496 -38.05281),212011550,Pakenham - North West,58924.808098,9820,Pakenham - North West,3.53,3.53,4.014286


In [128]:
gpt_rental_2024_house = gpd_rental_2024[gpd_rental_2024["type"] == "House"]
gpt_rental_2024_apmt = gpd_rental_2024[gpd_rental_2024["type"] == "Unit/apmt"]


In [134]:
gpt_rental_2024_apmt.groupby("suburb")[["rented_price","bed","bath","median_income","population","unemployment_rate"]].mean().reset_index().sort_values("rented_price", ascending=False).head(15)

,suburb,rented_price,bed,bath,median_income,population,unemployment_rate
477,MERRICKS BEACH,975.0,4.0,2.0,57312.685409,24384.0,4.014286
335,HERNES OAK,940.0,4.0,2.0,56019.600443,17585.0,4.014286
720,THE PATCH,920.0,4.0,2.0,65254.647043,9743.0,4.014286
441,MACEDON,900.0,4.0,2.0,77447.070501,3632.0,4.014286
264,FAIRBANK,895.0,3.0,1.0,51638.816175,11763.0,4.014286
447,MAIN RIDGE,895.0,4.0,2.0,52037.691013,6074.0,4.014286
603,QUANTONG,889.0,3.0,2.0,53392.175889,3380.0,4.014286
523,NARRACAN,875.0,3.5,2.0,55111.183940,8680.0,4.014286
533,NERRENA,875.0,4.0,2.0,51638.816175,11763.0,4.014286
476,MERRICKS,860.0,3.0,1.5,52037.691013,6074.0,4.014286


In [120]:
gpd_rental_2024["suburb"] = gpd_rental_2024["address"].apply(lambda x: x.split(", ")[-1])

In [124]:
gpd_rental_2024.to_csv("../data/curated/rental_data_2024.csv", index=False)